In [1]:
from datasets import load_dataset
swebench = load_dataset('princeton-nlp/SWE-bench', split='test')

/workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
print(dataset.column_names)

['repo', 'instance_id', 'base_commit', 'patch', 'test_patch', 'problem_statement', 'hints_text', 'created_at', 'version', 'FAIL_TO_PASS', 'PASS_TO_PASS', 'environment_setup_commit']


In [4]:
#Test code for docker setup

# !python -m swebench.harness.run_evaluation \
#     --predictions_path gold \
#     --max_workers 1 \
#     --instance_ids sympy__sympy-20590 \
#     --run_id validate-gold

<frozen runpy>:128: RuntimeWarning: 'swebench.harness.run_evaluation' found in sys.modules after import of package 'swebench.harness', but prior to execution of 'swebench.harness.run_evaluation'; this may result in unpredictable behaviour
Using gold predictions - ignoring predictions_path
README.md: 100%|███████████████████████████| 3.67k/3.67k [00:00<00:00, 29.1MB/s]
dev-00000-of-00001.parquet: 100%|████████████| 120k/120k [00:00<00:00, 5.87MB/s]
test-00000-of-00001.parquet: 100%|█████████| 1.12M/1.12M [00:00<00:00, 34.7MB/s]
Generating test split: 100%|████████| 300/300 [00:00<00:00, 25703.54 examples/s]
Running 1 unevaluated instances...
Building base image (sweb.base.x86_64:latest)
Base images built successfully.
Total environment images to build: 1
Building environment images: 100%|████████████████| 1/1 [01:11<00:00, 71.01s/it]
All environment images built successfully.
Running 1 instances...
100%|█████████████████████████████████████████████| 1/1 [00:47<00:00, 47.21s/it]
All inst

In [17]:
import json
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Example model, replace with your preferred one
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

def predict(inputs: dict):
    # Extract the repo and patch from the input dictionary
    repo = inputs.get('repo', '')
    patch = inputs.get('patch', '')

    # Prepare the input text by combining repo and patch details
    input_text = f"Repo: {repo}\nPatch:\n{patch}"

    # Tokenize the input text
    inputs_encoded = tokenizer.encode(input_text, return_tensors='pt')

    # Check if the input length exceeds the model's maximum length
    max_length = 2048
    if inputs_encoded.size(1) > max_length:
        # Truncate the input to fit within the max length
        inputs_encoded = inputs_encoded[:, :max_length]

    # Generate predictions (patch) using the model
    outputs = model.generate(inputs_encoded, max_length=2050, num_return_sequences=1)

    # Decode the generated patch text
    patch_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Return the instance_id and generated patch
    return {
        "instance_id": inputs['instance_id'],
        "model_patch": patch_text,  # This is the generated patch from the model
        "model_name_or_path": model_name
    }
# Define the number of predictions you want to generate
num_predictions = 3  # You can set this to the desired number of examples

# Initialize a list to store all the predictions
predictions = []

# Loop through the examples in the dataset and generate predictions
for i, example in enumerate(dataset):
    if i >= num_predictions:
        break  # Stop once the desired number of predictions is reached

    # Generate a prediction using the predict function
    result = predict(example)
    
    # Append the result to the predictions list
    predictions.append(result)

# Save the predictions to a JSON file
with open('predictions.json', 'w') as f:
    json.dump(predictions, f, indent=4)

Token indices sequence length is longer than the specified maximum sequence length for this model (2473 > 2048). Running this sequence through the model will result in indexing errors


In [1]:
!python -m swebench.harness.run_evaluation \
    --dataset_name princeton-nlp/SWE-bench \
    --predictions_path predictions.json \
    --max_workers 1 \
    --run_id "test"

2024-09-14 11:11:02,551 - datasets - INFO - PyTorch version 2.4.1 available.
<frozen runpy>:128: RuntimeWarning: 'swebench.harness.run_evaluation' found in sys.modules after import of package 'swebench.harness', but prior to execution of 'swebench.harness.run_evaluation'; this may result in unpredictable behaviour
README.md: 100%|███████████████████████████| 3.88k/3.88k [00:00<00:00, 21.4MB/s]
dev-00000-of-00001.parquet: 100%|██████████| 1.38M/1.38M [00:00<00:00, 32.0MB/s]
test-00000-of-00001.parquet: 100%|█████████| 12.1M/12.1M [00:00<00:00, 52.3MB/s]
train-00000-of-00001.parquet: 100%|███████████| 107M/107M [00:00<00:00, 257MB/s]
Generating train split: 100%|███| 19008/19008 [00:01<00:00, 14070.58 examples/s]
Running 3 unevaluated instances...
Base image sweb.base.x86_64:latest already exists, skipping build.
Base images built successfully.
Total environment images to build: 1
Building environment images: 100%|████████████████| 1/1 [01:27<00:00, 87.56s/it]
All environment images buil